<a href="https://colab.research.google.com/github/silaspc82/FIAP_IA_PARA_DEVS_TECH_CHALLENGE/blob/main/Tech_Challenge_Fase_04/FIAP_TechCallenge_Fase_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
!pip install opencv-python-headless deepface tf-keras tqdm deepface mediapipe face-recognition

In [27]:
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

In [34]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import os
from tqdm import tqdm
from deepface import DeepFace
import face_recognition
import numpy as np


def load_images_from_folder(folder):
    known_face_encodings = []
    known_face_names = []

    # Percorrer todos os arquivos na pasta fornecida
    for filename in os.listdir(folder):
        # Verificar se o arquivo é uma imagem
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Carregar a imagem
            image_path = os.path.join(folder, filename)
            image = face_recognition.load_image_file(image_path)
            # Obter as codificações faciais (assumindo uma face por imagem)
            face_encodings = face_recognition.face_encodings(image)

            if face_encodings:
                face_encoding = face_encodings[0]
                # Extrair o nome do arquivo, removendo o sufixo numérico e a extensão
                name = os.path.splitext(filename)[0][:-1]
                name = name.split("_")[0]
                # Adicionar a codificação e o nome às listas
                known_face_encodings.append(face_encoding)
                known_face_names.append(name)

    return known_face_encodings, known_face_names


mapa_indice_chave_mediapipe={}
mapa_indice_chave_mediapipe[0] = 'nose'
mapa_indice_chave_mediapipe[1] = 'left_eye_inner'
mapa_indice_chave_mediapipe[2] = 'left_eye'
mapa_indice_chave_mediapipe[3] = 'left_eye_outer'
mapa_indice_chave_mediapipe[4] = 'right_eye_inner'
mapa_indice_chave_mediapipe[5] = 'right_eye'
mapa_indice_chave_mediapipe[6] = 'right_eye_outer'
mapa_indice_chave_mediapipe[7] = 'left_ear'
mapa_indice_chave_mediapipe[8] = 'right_ear'
mapa_indice_chave_mediapipe[9] = 'mouth_left'
mapa_indice_chave_mediapipe[10] = 'mouth_right'
mapa_indice_chave_mediapipe[11] = 'left_shoulder'
mapa_indice_chave_mediapipe[12] = 'right_shoulder'
mapa_indice_chave_mediapipe[13] = 'left_elbow'
mapa_indice_chave_mediapipe[14] = 'right_elbow'
mapa_indice_chave_mediapipe[15] = 'left_wrist'
mapa_indice_chave_mediapipe[16] = 'right_wrist'
mapa_indice_chave_mediapipe[17] = 'left_pinky'
mapa_indice_chave_mediapipe[18] = 'right_pinky'
mapa_indice_chave_mediapipe[19] = 'left_index'
mapa_indice_chave_mediapipe[20] = 'right_index'
mapa_indice_chave_mediapipe[21] = 'left_thumb'
mapa_indice_chave_mediapipe[22] = 'right_thumb'
mapa_indice_chave_mediapipe[23] = 'left_hip'
mapa_indice_chave_mediapipe[24] = 'right_hip'
mapa_indice_chave_mediapipe[25] = 'left_knee'
mapa_indice_chave_mediapipe[26] = 'right_knee'
mapa_indice_chave_mediapipe[27] = 'left_ankle'
mapa_indice_chave_mediapipe[28] = 'right_ankle'
mapa_indice_chave_mediapipe[29] = 'left_heel'
mapa_indice_chave_mediapipe[30] = 'right_heel'
mapa_indice_chave_mediapipe[31] = 'left_foot_index'
mapa_indice_chave_mediapipe[32] = 'right_foot_index'


# Função para verificar movimentos
def coleta_posicao_landmarks(results_pose, mp_pose):
    lista_mapa_landmarks = []
    pose_landmarks_list = results_pose.pose_landmarks

    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]

        landmark_list = [ landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z, visibility=landmark.visibility, presence=landmark.presence) for landmark in pose_landmarks ]

        mapa_posicionamento = {}

        for idx_landmark_item, landmark_item in enumerate(landmark_list):
          mapa_posicionamento[mapa_indice_chave_mediapipe[idx_landmark_item]] = landmark_item

        lista_mapa_landmarks.append(mapa_posicionamento)

    return lista_mapa_landmarks


def converter_landmarks_numpy(results_pose, width, height):
    lista_current_landmarks = []
    pose_landmarks_list = results_pose.pose_landmarks

    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]

        current_landmarks = np.array(
            [[lm.x * width, lm.y * height] for lm in pose_landmarks]
        )
        lista_current_landmarks.append(current_landmarks)

    return lista_current_landmarks



# Mapear partes do corpo com landmarks
partes_corpo = {
    "cabeca": [0, 1, 2, 3, 4, 5, 6, 7],  # Nariz, olhos, orelhas
    "braco esquerdo": [11, 13, 15],  # Ombro, cotovelo, pulso esquerdo
    "braco direito": [12, 14, 16],  # Ombro, cotovelo, pulso direito
    "perna esquerda": [23, 25, 27],  # Quadril, joelho, tornozelo esquerdo
    "perna direita": [24, 26, 28],  # Quadril, joelho, tornozelo direito
}



pe_esquerdo_a_frente = False
braco_esquerdo_levantado = False
braco_direito_levantado = False

lista_posicionamento_anterior = None
def detecta_movimentos(results_pose, mp_pose, arquivo_relatorio_saida, cv2, frame, width, height, movement_threshold=22):
    global pe_esquerdo_a_frente
    global braco_esquerdo_levantado
    global braco_direito_levantado
    global partes_corpo
    global lista_posicionamento_anterior

    lista_mapa_posicionamento = coleta_posicao_landmarks(results_pose, mp_pose)
    lista_current_landmarks = converter_landmarks_numpy(results_pose, width, height)


    if not lista_posicionamento_anterior:
        lista_posicionamento_anterior = lista_current_landmarks

    for idx, (landmarks_corrente, landmarks_anterior, mapa_posicionamento_corrente) in enumerate(zip(lista_current_landmarks, lista_posicionamento_anterior, lista_mapa_posicionamento)):
        movimentos = []
        gestos = []
        cores = {}

        # Detectar movimento por parte do corpo
        for parte, indices in partes_corpo.items():
            deslocamento = np.mean(np.linalg.norm(landmarks_corrente[indices] - landmarks_anterior[indices], axis=1))
            if deslocamento > movement_threshold:
                membros_visiveis = True

                for indice in indices:
                    if mapa_posicionamento_corrente[mapa_indice_chave_mediapipe[indice]].visibility < 0.8:
                        membros_visiveis = False
                        break

                if membros_visiveis:
                    movimentos.append(parte)

        # Detectar gestos específicos
        if "braco direito" in movimentos:
            movimento_mao = np.linalg.norm(landmarks_corrente[16] - landmarks_corrente[12])
            if movimento_mao > movement_threshold * 2:
                gestos.append("Aceno de mao (direito)")

        if "braco esquerdo" in movimentos:
            movimento_mao = np.linalg.norm(landmarks_corrente[15] - landmarks_corrente[11])
            if movimento_mao > movement_threshold * 2:
                gestos.append("Aceno de mao (esquerdo)")

        if "perna direita" in movimentos and "perna esquerda" in movimentos:
            movimento_perna_direita = np.linalg.norm(landmarks_corrente[24] - landmarks_anterior[24])
            movimento_perna_esquerda = np.linalg.norm(landmarks_corrente[23] - landmarks_anterior[23])
            if abs(movimento_perna_direita - movimento_perna_esquerda) > movement_threshold:
                gestos.append("Caminhando")

        if movimentos:
            arquivo_relatorio_saida.write(f"\tMovimento {idx}: {movimentos}\n")

            cv2.putText(frame, f"Movimento {idx}: {movimentos}", (10, 30 + (idx * 60)),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        if gestos:
            arquivo_relatorio_saida.write(f"\tGestos {idx}: {gestos}\n")
            cv2.putText(frame, f"Gestos {idx}: {gestos}", (10, 60 + (idx * 60)),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    lista_posicionamento_anterior = lista_current_landmarks





def processa_video(video_path, output_path, exibir_frame_processado=False, diretorio_projeto="."):

    base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
    options = vision.PoseLandmarkerOptions(
        num_poses=10,
        base_options=base_options,
        output_segmentation_masks=True)
    detector = vision.PoseLandmarker.create_from_options(options)

    # Inicializar o MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(
        # min_detection_confidence=0.3,
        # min_tracking_confidence=0.3
    )
    mp_drawing = mp.solutions.drawing_utils

    # Capturar vídeo do arquivo especificado
    cap = cv2.VideoCapture(video_path)

    # Verificar se o vídeo foi aberto corretamente
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    # Obter propriedades do vídeo
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Definir o codec e criar o objeto VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    arquivo_relatorio_saida_caminho = diretorio_projeto + "relatorio_saida.txt"

    with open(arquivo_relatorio_saida_caminho, "w") as arquivo_relatorio_saida:

        # Gravando o total de frames do vídeo
        arquivo_relatorio_saida.write(
            f"""Inicio do processamento
  Total Frames: {total_frames}\n\n""")

        # Loop para processar cada frame do vídeo com barra de progresso
        for item_tqdm in tqdm(range(total_frames), desc="Processando vídeo"):

            # Ler um frame do vídeo
            ret, frame = cap.read()

            # Se não conseguiu ler o frame (final do vídeo), sair do loop
            if not ret:
                break

            # Gravando o índice de frame lido
            arquivo_relatorio_saida.write(
                f"\n\n\t----------\n\tFrame: {item_tqdm}\n")

            # Analisar o frame para detectar faces e expressões
            result_deepFace = DeepFace.analyze(
                frame, actions=['emotion'], enforce_detection=False)

            # Converter o frame para RGB
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Processar o frame para detectar a pose
            # results_pose = pose.process(rgb_frame)
            frame_to_detector = mp.Image(
                image_format=mp.ImageFormat.SRGB, data=rgb_frame)
            results_pose = detector.detect(frame_to_detector)

            face_locations = face_recognition.face_locations(
                rgb_frame, model="cnn")
            face_encodings = face_recognition.face_encodings(
                rgb_frame, face_locations)

            # Inicializar uma lista de nomes para as faces detectadas
            face_names = []
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(
                    known_face_encodings, face_encoding)
                name = "Desconhecido"
                face_distances = face_recognition.face_distance(
                    known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]
                face_names.append(name)

            # Iterar sobre cada face detectada
            for idx_face, face in enumerate(result_deepFace):
                # Obter a caixa delimitadora da face
                x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']

                # Obter a emoção dominante
                dominant_emotion = face['dominant_emotion']

                # Desenhar um retângulo ao redor da face
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                # Escrever a emoção dominante acima da face
                cv2.putText(frame, dominant_emotion, (x-50, y + h + 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

                arquivo_relatorio_saida.write(
                    f"\tExpressao dominante {idx_face}: {dominant_emotion}\n")

                # Associar a face detectada pelo DeepFace com as faces conhecidas
                for (top, right, bottom, left), name in zip(face_locations, face_names):
                    if x <= left <= x + w and y <= top <= y + h:
                        # Escrever o nome abaixo da face
                        cv2.putText(frame, name, (x - 50, y + h + 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
                        arquivo_relatorio_saida.write(f"\tFace reconhecida {idx_face}: {name} \t posicao: (top: {top}, right: {right}, bottom: {bottom}, left: {left})\n")
                        break

            # Desenhar as anotações da pose no frame
            if results_pose.pose_landmarks:
                lista_mapa_landmarks = []
                pose_landmarks_list = results_pose.pose_landmarks
                for idx in range(len(pose_landmarks_list)):
                    pose_landmarks = pose_landmarks_list[idx]
                    # Draw the pose landmarks.
                    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
                    pose_landmarks_proto.landmark.extend([
                        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
                    ])
                    mp_drawing.draw_landmarks(
                        frame, pose_landmarks_proto, mp_pose.POSE_CONNECTIONS)
                detecta_movimentos(
                    results_pose, mp_pose, arquivo_relatorio_saida, cv2, frame, width, height)

            # Escrever o frame processado no vídeo de saída
            out.write(frame)

            if exibir_frame_processado:
                # Exibir o frame processado
                cv2.imshow('Video', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                # Gravando o total de frames do vídeo
        arquivo_relatorio_saida.write(
            f"""\n\n\nFim do processamento
  Total Frames: {total_frames}\n\n""")

    # Liberar a captura de vídeo e fechar todas as janelas
    cap.release()
    out.release()
    if exibir_frame_processado:
        cv2.destroyAllWindows()


# Nova seção

In [29]:
caminho_projeto = '/content/drive/MyDrive/TI/Cursos/FIAP/IAParaDEVS/TechChallenge/Fase_04/'

In [30]:
# Caminho para a pasta de imagens com rostos conhecidos
image_folder = caminho_projeto + 'images'

# Carregar imagens e codificações
known_face_encodings, known_face_names = load_images_from_folder(image_folder)

In [31]:
# Caminho para o vídeo de entrada e saída
input_video_path = os.path.join(caminho_projeto, 'Unlocking Facial Recognition_ Diverse Activities Analysis.mp4')  # Nome do vídeo de entrada
output_video_path = os.path.join(caminho_projeto, 'output_detection_and_recognition_Unlocking Facial Recognition_ Diverse Activities Analysis.mp4')  # Nome do vídeo de saída

In [35]:
# Processar o vídeo
processa_video(input_video_path, output_video_path, exibir_frame_processado=False, diretorio_projeto=caminho_projeto)

Processando vídeo: 100%|██████████| 3326/3326 [24:51<00:00,  2.23it/s]
